# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
!jupyter nbextension enable --py gmaps


In [ ]:
%matplotlib inline

In [ ]:
!pip install gmaps

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
city_data_to_load = "citydata.xlsx"
city_data = pd.read_csv(city_data_to_load)


city_data[['Latitude', 'Longitude']].dtypes



Latitude     float64
Longitude    float64
dtype: object

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
gmaps.configure(api_key=g_key)

In [7]:
locations = city_data[["Latitude", "Longitude"]]
humidity = city_data["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=200, point_radius=5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
idealtemp_df=city_data.loc[city_data['Max Temp F'] >60]
idealtemp_df=city_data.loc[city_data['Max Temp F'] <75]
idealwind_df=idealtemp_df.loc[idealtemp_df["Wind Speed mph"] <10]
idealweather_df=idealwind_df.loc[idealwind_df["Humidity"]<40]


idealweather_df.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Wind Speed,Country,Cloudiness,Date,Max Temp F,Wind Speed mph
47,47,Nikolskoye,59.70,30.79,14.00,30,4.00,RU,33,1590348252,57.20,8.95
55,55,Kumul,42.80,93.45,20.60,30,2.83,CN,100,1590348324,69.08,6.33
96,96,Central Point,42.38,-122.92,21.67,37,3.10,US,1,1590348260,71.01,6.93
117,117,Khorixas,-20.37,14.97,19.81,29,1.48,NaN,0,1590348066,67.66,3.31
134,134,Saint George,37.10,-113.58,23.00,12,4.10,US,1,1590348112,73.40,9.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
hotel_df=idealweather_df
hotel_df["Hotel Name"] = ""



hotel_df.head()

C:\Users\david\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Wind Speed,Country,Cloudiness,Date,Max Temp F,Wind Speed mph,Hotel Name
47,47,Nikolskoye,59.70,30.79,14.00,30,4.00,RU,33,1590348252,57.20,8.95,
55,55,Kumul,42.80,93.45,20.60,30,2.83,CN,100,1590348324,69.08,6.33,
96,96,Central Point,42.38,-122.92,21.67,37,3.10,US,1,1590348260,71.01,6.93,
117,117,Khorixas,-20.37,14.97,19.81,29,1.48,NaN,0,1590348066,67.66,3.31,
134,134,Saint George,37.10,-113.58,23.00,12,4.10,US,1,1590348112,73.40,9.17,


In [13]:
params = {
    'radius': 5000,
    'types': 'lodging',
    'key': g_key
}

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

C:\Users\david\anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Wind Speed,Country,Cloudiness,Date,Max Temp F,Wind Speed mph,Hotel Name
47,47,Nikolskoye,59.70,30.79,14.00,30,4.00,RU,33,1590348252,57.20,8.95,Tourist House - Sablino
55,55,Kumul,42.80,93.45,20.60,30,2.83,CN,100,1590348324,69.08,6.33,Laoyutang Resort
96,96,Central Point,42.38,-122.92,21.67,37,3.10,US,1,1590348260,71.01,6.93,Courtyard by Marriott Medford Airport
117,117,Khorixas,-20.37,14.97,19.81,29,1.48,NaN,0,1590348066,67.66,3.31,Khorixas Lodge
134,134,Saint George,37.10,-113.58,23.00,12,4.10,US,1,1590348112,73.40,9.17,Best Western Plus Abbey Inn
153,153,Khovd,48.01,91.64,13.97,34,1.02,MN,0,1590348330,57.15,2.28,Hotel Khovd
220,220,Palmer,42.16,-72.33,22.78,28,2.60,US,40,1590348334,73.00,5.82,Wedgewood Motel
239,239,Kajaani,64.23,27.73,15.00,28,1.50,FI,0,1590348336,59.00,3.36,Original Sokos Hotel Valjus
276,276,Gilgit,35.92,74.31,21.65,28,2.20,PK,16,1590348338,70.97,4.92,Tourist Hamlet Gilgit
353,353,East London,-33.02,27.91,21.88,36,3.14,ZA,0,1590348343,71.38,7.02,Tu Casa


In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

locations=pd.DataFrame(locations, columns=["Latitude", 
                                             "Longitude"])
len(locations)

18

In [15]:
# Add marker layer ontop of heat map

marker_locations = locations[["Latitude", "Longitude"]]
locations = city_data[["Latitude", "Longitude"]]
humidity = city_data["Humidity"]


fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=200, point_radius=5)

fig.add_layer(heat_layer)

markers = gmaps.marker_layer(marker_locations)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))